# 📦 PSMC Preprocessing Workflow

This notebook prepares the final species list with estimated mutation rates (`mu`) and generation times (`gen`) for the demographic inference pipeline.

- **R section:** Resolves phylogenetic relationships using Open Tree of Life
- **Python section:** Merges metadata, relative-based parameters, and fallback group averages


In [8]:
# Load libraries
library(readr)
library(caper)
library(ape)
library(treeio)
library(janitor)
library(rotl)
library(tidyverse)

In [ ]:
wang_species <- read_lines("files/wang_species.txt", locale = locale(encoding = "UTF-8")) %>%
  str_replace_all("_", " ") %>%
  str_trim()

my_species <- read_lines("files/my_species.txt") %>%
  str_replace_all("_", " ") %>%
  str_trim()

wang_df <- tibble(species = wang_species, project = "wang")
sv_df   <- tibble(species = my_species, project = "sv_div")
all_species_df <- bind_rows(wang_df, sv_df)

synonym_map <- c(
  "Aegotheles albertisi" = "Aegotheles albertisi albertisi",
  "Amazona ochrocephala" = "Amazona ochrocephala ochrocephala",
  "Ammospiza maritima maritima" = "Ammospiza maritima",
  "Ammospiza nelsoni" = "Ammodramus nelsoni",
  "Aspidoscelis tigris stejnegeri" = "Aspidoscelis tigris",
  "Callipepla californica brunnescens" = "Callipepla californica",
  "Coluber constrictor foxii" = "Coluber constrictor",
  "Glossophaga mutica" = "Glossophaga soricina",
  "Hydrolagus colliei" = "Hydrolagus",
  "Malaclemys terrapin pileata" = "Malaclemys terrapin",
  "Miniopterus schreibersii" = "Miniopterus schreibersii schreibersii",
  "Molossus nigricans" = "Molossus rufus",
  "Neogale vison" = "Neovison vison",
  "Osmerus mordax" = "Osmerus mordax mordax",
  "Rhamphochromis sp. 'chilingali'" = "Rhamphochromis",
  "Thamnophis sirtalis fitchi" = "Thamnophis sirtalis",
  "Tupaia chinensis belangeri" = "Tupaia belangeri"
)

all_species_df <- all_species_df %>%
  mutate(ott_name = recode(species, !!!synonym_map, .default = species))

name_resolution <- tnrs_match_names(all_species_df$ott_name) %>% clean_names()

tree_df <- all_species_df %>%
  mutate(ott_name_lower = tolower(ott_name)) %>%
  left_join(name_resolution %>% select(search_string, unique_name, ott_id),
            by = c("ott_name_lower" = "search_string")) %>%
  select(-ott_name_lower) %>%
  mutate(tree_label = paste0(str_replace_all(unique_name, " ", "_"), "_ott", ott_id)) %>%
  rename(original_name = species)

combined_tree <- tol_induced_subtree(ott_ids = tree_df$ott_id)

otol_tree_tibble <- as_tibble(combined_tree) %>%
  left_join(tree_df %>% select(tree_label, original_name, ott_name, project),
            by = c("label" = "tree_label"))

find_closest_wang_clade <- function(species_name, tree, tree_tibble) {
  message("Matching: ", species_name)
  
  node_info <- tree_tibble %>%
    filter(ott_name == species_name, project == "sv_div")
  
  is_self_match <- tree_tibble %>%
    filter(ott_name == species_name, project == "wang") %>%
    nrow() > 0
  
  if (is_self_match) {
    original_wang_species <- tree_tibble %>%
      filter(ott_name == species_name, project == "wang") %>%
      pull(original_name) %>% .[1]
    
    return(tibble(focus_species = species_name,
                  closest_relative = original_wang_species,
                  match_type = "self"))
  }

  tip_node <- node_info$node
  current_node <- node_info$parent
  found <- FALSE

  while (!found && !is.na(current_node)) {
    clade_members <- tryCatch({
      caper::clade.members(current_node, tree)
    }, error = function(e) NA)

    if (is.na(clade_members)[1]) break

    match <- tree_tibble %>%
      filter(node %in% clade_members[clade_members != tip_node],
             project == "wang") %>%
      slice_min(order_by = node, with_ties = FALSE)

    if (nrow(match) > 0) {
      return(tibble(focus_species = species_name,
                    closest_relative = match$original_name,
                    match_type = "clade"))
    }

    current_node <- tree_tibble %>%
      filter(node == current_node) %>%
      pull(parent)
  }
}

# Apply the tree traversal to match SV species to Wang species
species_to_match <- tree_df %>% filter(project == "sv_div") %>% pull(ott_name)

closest_relative_df <- map_dfr(species_to_match, ~ find_closest_wang_clade(.x, combined_tree, otol_tree_tibble))

final_matches <- closest_relative_df %>%
  mutate(scientific_name = my_species,
         ott_name = focus_species) %>%
  select(scientific_name, ott_name, closest_relative)

write_csv(final_matches, "files/wang_relatives.csv")


Warning message:
"Some names were duplicated: 'pan troglodytes', 'betta splendens', 'neovison vison', 'pungitius pungitius', 'vulpes vulpes', 'orcinus orca', 'taeniopygia guttata', 'homo sapiens', 'arctocephalus gazella', 'ornithorhynchus anatinus', 'pongo abelii', 'tursiops truncatus', 'callithrix jacchus', 'panthera pardus', 'thamnophis sirtalis', 'pan troglodytes', 'macaca mulatta', 'aotus nancymaae', 'monodelphis domestica', 'rhea pennata', 'pelecanus crispus', 'tapirus indicus', 'microcebus murinus', 'capra hircus'."
Warning message in FUN(X[[i]], ...):
"unable to translate '<U+00C4>' to native encoding"
Warning message in FUN(X[[i]], ...):
"unable to translate '<U+00D6>' to native encoding"
Warning message in FUN(X[[i]], ...):
"unable to translate '<U+00DC>' to native encoding"
Warning message in FUN(X[[i]], ...):
"unable to translate '<U+00E4>' to native encoding"
Warning message in FUN(X[[i]], ...):
"unable to translate '<U+00F6>' to native encoding"
Warning message in FUN(X[[i

In [ ]:
import pandas as pd

assemblies = pd.read_csv("downloaded_assemblies.tsv", sep="\t")
autosomes = pd.read_csv("autosomes_by_accession.tsv", sep="\t", names=["accession", "autosomes"])
metadata = pd.read_csv("assembly_metadata_wide.tsv", sep="\t")
wang_relatives = pd.read_csv("wang_relatives.csv")

wang_mu = pd.read_excel("mutation_rate_literature_updating3.xlsx", sheet_name="raw")
wang_gen = pd.read_excel("mutation_rate_literature_updating3.xlsx", sheet_name="gs_gt")

df = assemblies.merge(autosomes, on="accession", how="left")
df = df[df["assembly_category"] == "pri"]
df = df.merge(metadata[['scientific_name', 'taxonomic_group']], how='left', on='scientific_name')
df = df.merge(wang_relatives[['scientific_name', 'closest_relative']], how='inner', on='scientific_name')

# Mutation rates (filtered to snps + vertebrates)
wang_mu_snps = wang_mu[(wang_mu['TYPE'] == 'snp') &
                       (wang_mu['Group'].isin(['human', 'fish', 'primates', 'mammals', 'birds', 'reptiles']))]

mu_avgs = wang_mu_snps.groupby('Species', as_index=False)['u_mean'].mean()

df = df.merge(mu_avgs, left_on='closest_relative', right_on='Species', how='left').drop(columns='Species').rename(columns={'u_mean': 'mu'})
df = df.merge(wang_gen[['Species', 'Gt_Year']], left_on='closest_relative', right_on='Species', how='left').drop(columns='Species').rename(columns={'Gt_Year': 'gen'})

missing_gens = {
    'Pungitius pungitius': 1.5,
    'Poecilia reticulata': 0.6,
    'Neogale vison': 1
}

df.loc[df['taxonomic_group'] == 'amphibian', ['mu']] = 0.776e-9
df['gen'] = df['gen'].fillna(df['closest_relative'].map(missing_gens))

avg_mu_by_group = {
    'reptile': 12.41e-9,
    'bird': 10.43e-9,
    'mammal': 8.36e-9,
    'ray-finned fish': 5.55e-9,
    'cartilaginous fish': 5.55e-9,
    'jawless fish': 5.55e-9,
    'amphibian': 0.776e-9
}
df["group_mu"] = df["taxonomic_group"].map(avg_mu_by_group)

final = df[['scientific_name', 'assembly_bio_sample_accession', 'accession', 'fasta', 'mu', 'gen', 'group_mu', 'autosomes']].rename(columns={'assembly_bio_sample_accession': 'biosample'})
final['scientific_name'] = final['scientific_name'].str.replace(' ', '_')
final.to_csv("files/final_species_list.tsv", sep="\t", index=False)